<a href="https://colab.research.google.com/github/ymoslem/datasets/blob/main/LongContextQE/Long-Context-MT-QE-WMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install datasets polars -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 21.6 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("ymoslem/wmt-da-human-evaluation",
                       )

dataset

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.92M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1275190 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['lp', 'src', 'mt', 'ref', 'score', 'raw', 'annotators', 'domain', 'year'],
        num_rows: 1275190
    })
    test: Dataset({
        features: ['lp', 'src', 'mt', 'ref', 'score', 'raw', 'annotators', 'domain', 'year'],
        num_rows: 10000
    })
})

In [3]:
df_train = dataset["train"].to_polars().drop_nulls()
df_test = dataset["test"].to_polars().drop_nulls()

print(f"Train: {df_train.shape[0]:,} Test: {df_test.shape[0]:,}")
print(f"Domains: {df_train['domain'].unique().to_list()}")

Train: 1,275,180 Test: 10,000
Domains: ['conversation', 'news', 'ecommerce', 'other', 'wiki', 'social']


In [4]:
import polars as pl
from random import randrange
from tqdm.auto import tqdm


def pair_sentences(df):
  df1 = df.with_row_index()
  df2 = df1.sample(fraction=1, shuffle=True)

  group_columns = ["lp", "domain"]

  # Group by language and domain
  df1_groups = df1.group_by(group_columns).agg(pl.all()).sort(group_columns)
  df2_groups = df2.group_by(group_columns).agg(pl.all()).sort(group_columns)

  df_joined = df1_groups.join(df2_groups,
                              on=group_columns,
                              how="left",
                              suffix="_pair")

  df_paired = df_joined.explode(pl.exclude(group_columns))

  # Filter out exact indices
  df_paired = df_paired.filter(pl.col("index") != pl.col("index_pair"))

  # Combine the sentences in each pair
  df_paired = df_paired.with_columns([
      (pl.col("src") + " " + pl.col("src_pair")).alias("src_doc"),
      (pl.col("ref") + " " + pl.col("ref_pair")).alias("ref_doc"),
      (pl.col("mt") + " " + pl.col("mt_pair")).alias("mt_doc"),
  ])

  # Compute weighted average score
  df_scored = df_paired.with_columns(
      # Compute word counts
      (
          pl.col("src").str.len_chars() + pl.col("mt").str.len_chars()
      ).alias("char_count_1"),
      (
          pl.col("src_pair").str.len_chars() + pl.col("mt_pair").str.len_chars()
      ).alias("char_count_2")
  ).with_columns(
      # Weighted score by character count
      (
          (pl.col("char_count_1") * pl.col("raw") + pl.col("char_count_2") * pl.col("raw_pair"))
          /
          (pl.col("char_count_1") + pl.col("char_count_2"))
      ).alias("raw_doc")
  )

  columns_to_keep = ['lp','src_doc', 'mt_doc', 'ref_doc', 'raw_doc', 'domain', 'year']
  df_scored = df_scored.select(columns_to_keep)
  df_scored = df_scored.rename({"src_doc": "src",
                                "mt_doc": "mt",
                                "ref_doc": "ref",
                                "raw_doc": "raw"})

  return df_scored


def process(df_scored, iterations):

  df_scored = df_scored.with_columns(sents = 1)
  df_scored = df_scored.drop("score", "annotators")

  dfs = [df_scored]
  count = 1

  for i in tqdm(range(iterations)):
    df_scored = pair_sentences(df_scored)
    count += count
    df_scored = df_scored.with_columns(sents = count)
    dfs.append(df_scored)

  dfs_scored_concatenated = pl.concat(dfs)

  return dfs_scored_concatenated

In [5]:
# create the long-context train dataset
df_scored_train = process(df_train, 5)

  0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
# create the long-context test dataset
df_scored_test = process(df_test, 5)

# show 3 random rows
with pl.Config(set_fmt_str_lengths=300):
  display(df_scored_test.sample(n=3))

  0%|          | 0/5 [00:00<?, ?it/s]

lp,src,mt,ref,raw,domain,year,sents
str,str,str,str,f64,str,i64,i32
"""de-en""","""Einen passenden Ersatzakku können Sie über unseren Webshop beziehen und so die Nutzungsdauer Ihrer Dekoleuchte noch weiter verlängern. Wir senden Dir eine E-Mail mit der Sendungsnummer, sofern Du DHL oder DPD als Versender ausgewählt hast. Wenn die Spieler genügend Revolverhelden beigesteuert hab…","""You can obtain a suitable replacement battery from our webshop and thus extend the service life of your decorative light even further. We will send you an email with the tracking number if you have selected DHL or DPD as the sender. If the players have contributed enough revolving heroes and are sup…","""You can find the right replacement batteries in our online shop and thereby further lengthen the usage life of your decorative light. We send you an email with the shipping number as long as you chose DHL or DPD for shipping. If the players have contributed enough gunslingers and they are numericall…",0.649571,"""ecommerce""",2022,32
"""en-de""","""""That is perhaps a reason why e-cigarette use remains relatively low in Scotland. Ford also testified at the hearing about her allegation. Pruitt presented the change as his agency being more responsive than past administrations to the needs of state environmental regulators. The stamp duty rise wil…","""„Das ist vielleicht ein Grund, warum der E-Zigarettenkonsum in Schottland nach wie vor relativ gering ist. Ford sagte auch bei der Anhörung über ihre Behauptung aus. Pruitt präsentierte die Änderung als seine Agentur, die auf die Bedürfnisse der staatlichen Umweltaufsichtsbehörden besser reagiert al…","""„Das ist möglicherweise ein Grund dafür, dass der Gebrauch von E-Zigaretten in Schottland bisher recht wenig verbreitet ist.“ Ford sagte auch bei der Anhörung über die Anschuldigung aus. Pruitt präsentierte den Wandel, als seine Agentur sich den Bedürfnissen der staatlichen Umweltbehörden gegenüber …",0.945065,"""news""",2020,8
"""ta-en""","""ஆனால் ஊழல் வழக்கு ஒன்றில் 2016-ம் ஆண்டு தமாங்குக்கு கீழ்நீதிமன்றம் சிறை தண்டனை விதித்தது. சேரன், சோழன், பாண்டியன், சாளுக்கியர்கள், கிருஷ்ண தேவ ராயர் உள்பட பல மன்னர்கள் ஆட் சிக் காலத்தில் பெண்களை படிக்கச் சொல்லவில்லை. இதுதவிர நிலநடுக்கம் காரண மாக 100 பேர் காயமடைந்த னர்; 2,000 பேர் பாதுகாப்பான பகுதிகள…","""However, in 2016, a lower court sentenced Tamang to life imprisonment in a corruption case. Many kings, including Cheran, Chola, Pandiyan, Chalukyas and Krishna Deva Royer, did not ask women to learn during the Ad Chic period. In addition, 100 people were injured as an earthquake, killing 2,000 peop…","""But Damang was sentenced to prison in 2016 by a lower court in a corruption case. During the rule of many kings including Chera, Chola, Pandya, Chalukya and Krishnadeva Raya, women were not told to study. Apart from this 100 people were injured due to earthquake; 2,000 persons were shifted to safe a…",0.653722,"""news""",2020,32


In [8]:
# Verify there are no nulls
print(df_scored_train.null_count())
print(df_scored_test.null_count())

shape: (1, 8)
┌─────┬─────┬─────┬─────┬─────┬────────┬──────┬───────┐
│ lp  ┆ src ┆ mt  ┆ ref ┆ raw ┆ domain ┆ year ┆ sents │
│ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ ---    ┆ ---  ┆ ---   │
│ u32 ┆ u32 ┆ u32 ┆ u32 ┆ u32 ┆ u32    ┆ u32  ┆ u32   │
╞═════╪═════╪═════╪═════╪═════╪════════╪══════╪═══════╡
│ 0   ┆ 0   ┆ 0   ┆ 0   ┆ 0   ┆ 0      ┆ 0    ┆ 0     │
└─────┴─────┴─────┴─────┴─────┴────────┴──────┴───────┘
shape: (1, 8)
┌─────┬─────┬─────┬─────┬─────┬────────┬──────┬───────┐
│ lp  ┆ src ┆ mt  ┆ ref ┆ raw ┆ domain ┆ year ┆ sents │
│ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ ---    ┆ ---  ┆ ---   │
│ u32 ┆ u32 ┆ u32 ┆ u32 ┆ u32 ┆ u32    ┆ u32  ┆ u32   │
╞═════╪═════╪═════╪═════╪═════╪════════╪══════╪═══════╡
│ 0   ┆ 0   ┆ 0   ┆ 0   ┆ 0   ┆ 0      ┆ 0    ┆ 0     │
└─────┴─────┴─────┴─────┴─────┴────────┴──────┴───────┘


# Create HF Dataset

In [9]:
# Create dataset from Polars dataframe
from datasets import Dataset, DatasetDict

dataset_train = Dataset.from_polars(df_scored_train)
dataset_test = Dataset.from_polars(df_scored_test)

dataset = DatasetDict({
    "train": dataset_train,
    "test": dataset_test
})

dataset

DatasetDict({
    train: Dataset({
        features: ['lp', 'src', 'mt', 'ref', 'raw', 'domain', 'year', 'sents'],
        num_rows: 7650287
    })
    test: Dataset({
        features: ['lp', 'src', 'mt', 'ref', 'raw', 'domain', 'year', 'sents'],
        num_rows: 59235
    })
})

In [12]:
commit_message = "WMT long-context DA eval"
commit_description="WMT DA human evaluation dataset augmented into 2, 4, 8, 16, and 32 sentences"

user = "user_name"  # change

dataset.push_to_hub(f"{user}/wmt-da-human-evaluation-long-context",
                    data_dir="data",
                    private=True,
                    commit_message=commit_message,
                    commit_description=commit_description
                    )

Uploading the dataset shards:   0%|          | 0/74 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/104 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/104 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/104 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/104 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/104 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/60 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ymoslem/wmt-da-human-evaluation-long-context/commit/38ddbdc9cd917c60795561db8fdfdf75fb6d1739', commit_message='WMT long-context DA eval (part 00001-of-00002)', commit_description='WMT DA human evaluation dataset augmented into 2, 4, 8, 16, and 32 sentences', oid='38ddbdc9cd917c60795561db8fdfdf75fb6d1739', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ymoslem/wmt-da-human-evaluation-long-context', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ymoslem/wmt-da-human-evaluation-long-context'), pr_revision=None, pr_num=None)